After download the data using python scripts('nyc-motor-collisions.py'), we will use this notebook to perform an analysis of the data.

### 1. Import the package and downloaded datasets onto the workspace.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('nyc-collisions.csv')
data.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,...,vehicle_type_code1,contributing_factor_vehicle_2,vehicle_type_code2,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,cross_street_name,contributing_factor_vehicle_5,vehicle_type_code_5
0,2024-12-06T00:00:00.000,14:16,QUEENS,11367.0,40.742200,-73.829880,"{'latitude': '40.7422', 'longitude': '-73.8298...",61 RD,136 ST,0,...,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-12-06T00:00:00.000,9:20,BRONX,10475.0,40.890076,-73.819850,"{'latitude': '40.890076', 'longitude': '-73.81...",BOSTON RD,ROPES AVE,1,...,Sedan,Unspecified,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-06T00:00:00.000,6:10,BROOKLYN,11221.0,40.686367,-73.929750,"{'latitude': '40.686367', 'longitude': '-73.92...",MALCOLM X BLVD,PUTNAM AVE,2,...,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN
3,2024-12-06T00:00:00.000,16:30,BROOKLYN,11214.0,40.608000,-74.001720,"{'latitude': '40.608', 'longitude': '-74.00172...",18 AVE,NEW UTRECHT AVE,0,...,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,Bike,NaN,NaN,NaN
4,2024-12-06T00:00:00.000,16:48,QUEENS,11421.0,40.697560,-73.852745,"{'latitude': '40.69756', 'longitude': '-73.852...",WOODHAVEN BLVD,PARK LN S,1,...,E-Bike,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# since 'location' column contains the dictionary of location whcih is already stated in other columns, we will remove it from our dataframe for more memory-efficient of work.
data.drop(['location'],axis=1,inplace=True)

In [5]:
data.shape

(285147, 28)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285147 entries, 0 to 285146
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   crash_date                     285147 non-null  object 
 1   crash_time                     285147 non-null  object 
 2   borough                        194413 non-null  object 
 3   zip_code                       194375 non-null  float64
 4   latitude                       261913 non-null  float64
 5   longitude                      261913 non-null  float64
 6   on_street_name                 206109 non-null  object 
 7   off_street_name                138711 non-null  object 
 8   number_of_persons_injured      285147 non-null  int64  
 9   number_of_persons_killed       285147 non-null  int64  
 10  number_of_pedestrians_injured  285147 non-null  int64  
 11  number_of_pedestrians_killed   285147 non-null  int64  
 12  number_of_cyclist_injured     

In [27]:
data.isnull().sum()

crash_date                            0
crash_time                            0
borough                           90734
zip_code                          90772
latitude                          23234
longitude                         23234
on_street_name                    79038
off_street_name                  146436
number_of_persons_injured             0
number_of_persons_killed              0
number_of_pedestrians_injured         0
number_of_pedestrians_killed          0
number_of_cyclist_injured             0
number_of_cyclist_killed              0
number_of_motorist_injured            0
number_of_motorist_killed             0
contributing_factor_vehicle_1      1852
collision_id                          0
vehicle_type_code1                 4442
contributing_factor_vehicle_2     66644
vehicle_type_code2                96057
contributing_factor_vehicle_3    258713
vehicle_type_code_3              260637
contributing_factor_vehicle_4    278104
vehicle_type_code_4              278511


### 2. Exploratory Data Analysis

In [7]:
# find out the common cuase for vehicle collision
# concatenate the collision cause into one place
collisioncause = pd.concat([
    data['contributing_factor_vehicle_1'],
    data['contributing_factor_vehicle_2'],
    data['contributing_factor_vehicle_3'],
    data['contributing_factor_vehicle_4'],
    data['contributing_factor_vehicle_5']
], axis=0, ignore_index=True) 

In [25]:
collisioncause.unique()

array(['Driver Inattention/Distraction', 'Aggressive Driving/Road Rage',
       'Traffic Control Disregarded', 'Alcohol Involvement',
       'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
       'Following Too Closely', 'Turning Improperly', 'Oversized Vehicle',
       'Failure to Yield Right-of-Way', 'Physical Disability',
       'Other Vehicular', 'Unsafe Speed', 'Driver Inexperience',
       'Cell Phone (hand-Held)', 'View Obstructed/Limited',
       'Passing Too Closely', 'Unsafe Lane Changing', nan,
       'Pavement Slippery', 'Steering Failure', 'Backing Unsafely',
       'Fell Asleep', 'Reaction to Uninvolved Vehicle',
       'Passing or Lane Usage Improper', 'Glare', 'Brakes Defective',
       'Driverless/Runaway Vehicle', 'Lost Consciousness',
       'Fatigued/Drowsy', 'Obstruction/Debris', 'Passenger Distraction',
       'Outside Car Distraction', 'Illnes', 'Accelerator Defective',
       'Tire Failure/Inadequate', 'Using On Board Navigation Device',
       'Animals

In [19]:
# filter the values
collisioncause = collisioncause[collisioncause != 'Unspecified']

In [ ]:
collisioncause.value_counts()

Driver Inattention/Distraction                           83556
Following Too Closely                                    21842
Failure to Yield Right-of-Way                            21747
Passing or Lane Usage Improper                           14664
Unsafe Speed                                             12329
Other Vehicular                                          12260
Passing Too Closely                                      11224
Traffic Control Disregarded                               9681
Backing Unsafely                                          9124
Turning Improperly                                        7332
Driver Inexperience                                       6733
Unsafe Lane Changing                                      6625
Alcohol Involvement                                       5460
Pedestrian/Bicyclist/Other Pedestrian Error/Confusion     3914
Reaction to Uninvolved Vehicle                            3902
View Obstructed/Limited                                

In [23]:
# then, we will find out the most common vehicle type to be collided.
collisionvehicle = pd.concat([
    data['vehicle_type_code1'],
    data['vehicle_type_code2'],
    data['vehicle_type_code_3'],
    data['vehicle_type_code_4'],
    data['vehicle_type_code_5']
], axis=0, ignore_index=True) 

In [26]:
collisionvehicle.value_counts()

Sedan                                  222738
Station Wagon/Sport Utility Vehicle    169601
Bike                                    16207
Pick-up Truck                           12465
Taxi                                    11408
                                        ...  
vAN                                         1
u haul                                      1
HURSE                                       1
FDNY285 En                                  1
SALTSPREAD                                  1
Name: count, Length: 1257, dtype: int64

In [35]:
# find out the most common collision cause by vehicle type
data.groupby(['vehicle_type_code1','contributing_factor_vehicle_1'])['collision_id'].count().sort_values(ascending=False)

vehicle_type_code1                   contributing_factor_vehicle_1 
Sedan                                Unspecified                       34218
                                     Driver Inattention/Distraction    31755
Station Wagon/Sport Utility Vehicle  Driver Inattention/Distraction    25064
                                     Unspecified                       22987
Sedan                                Failure to Yield Right-of-Way      8161
                                                                       ...  
truck                                Reaction to Uninvolved Vehicle        1
                                     Turning Improperly                    1
unk                                  Unsafe Speed                          1
                                     Unspecified                           1
us postal                            Other Vehicular                       1
Name: collision_id, Length: 2577, dtype: int64

In [36]:
# find out the most frequent collision site (site is defined by borough)
data['borough'].value_counts()

borough
BROOKLYN         67197
QUEENS           52663
MANHATTAN        34650
BRONX            32000
STATEN ISLAND     7903
Name: count, dtype: int64

In [44]:
# which type of vehicle is the safest type
data.groupby(['vehicle_type_code1'])['collision_id'].count()

vehicle_type_code1
.              1
0000           1
10 Paaseng     1
100th truc     1
13             1
              ..
van           54
van truck      1
wag            1
wagon          2
ХР06           1
Name: collision_id, Length: 732, dtype: int64